## VTT formatter

This VTT formatter takes a WEBVTT transcript and creates a formatted text file which contains the messages without the time stamps and identifiers. 

The `.vtt` file is read into the `VttFormatter` class using the `import_file` method. This opend the file and appends each line in the file to a list.
This list is looped over and each line is added to a dictionary depending on the information contained in the line using the `create_dictionary` method. 
The text is the reformatted to ensure the text flows conversationally based on the start/stop time stamps contained in the original file using the `format_text` method and the reformatted text is written to a new `.txt` file using the name of the original file in the `reformat_vtt` method.

The simplest way to reformat a `.vtt` file to a `.txt` file is to use the following:

```python
from vttformatter.vttformatter import VttFormatter
filename = 'path_to_file.vtt'
vtt = VttFormatter(filename)
vtt.reformat_vtt()
```

In [1]:
from vttformatter.vttformatter import VttFormatter

In [2]:
# define the path to the original WEBVTT file
# the updated .txt file will be saved in the same directory as the original
filename = 'test_AutoGeneratedCaption.vtt'

In [3]:
# create a VttFormatter object
vtt = VttFormatter(filename)
# use this method to automatically reformat the .vtt and save the new file
vtt.reformat_vtt()

In [4]:
# this code creates a dictionary for each element in the original file
# the dictionary is stored as an attribute which can be accessed
vtt.data_dict

{'messages': [{'confidence': '0.69450831413269',
   'marker': 'ef04c7c2-a59e-463f-9d27-b5b1259d6777',
   'start': '00:00:03.300',
   'stop': '00:00:06.870',
   'content': ['Sorry.']},
  {'confidence': '0.621036410331726',
   'marker': '8a017ebb-1722-4e7f-8984-fc6da39c3489',
   'start': '00:00:08.100',
   'stop': '00:00:09.620',
   'content': ['Yes.']},
  {'confidence': '0.713402450084686',
   'marker': 'd9a1567a-1ebe-40ce-983a-98436bcabcfe',
   'start': '00:00:19.240',
   'stop': '00:00:20.240',
   'content': ['Picking up your feet.']},
  {'confidence': '0.536461710929871',
   'marker': 'b8e0fa64-8c2f-4070-9b0f-922a50f3fcde',
   'start': '00:00:21.930',
   'stop': '00:00:23.490',
   'content': ['Yeah.']},
  {'confidence': '0.889019846916199',
   'marker': '88910870-8af9-48f5-bcc4-a501eda95d3f',
   'start': '00:00:24.670',
   'stop': '00:00:28.778',
   'content': ['But now my headphones or what',
    'the boy, I could it still yeah']},
  {'confidence': '0.889019846916199',
   'marker': 

In [5]:
# the reformatted messages are stored as an attribute which can be accessed
vtt.full_messages

['Sorry.',
 'Yes.',
 'Picking up your feet.',
 'Yeah.',
 "But now my headphones or what the boy, I could it still yeah well. I mean, it's it. It will still be recording the audio. It's just not playing in through the speakers to see something in there, if you click. The 3:00 dots. I saw something that said turn off incoming video so that would turn off.",
 "10s of me, so I don't know if that.",
 'Yes, yeah, yeah, so you can actually control how you want the recording to go you can.']